In [2]:
# READING PACKAGES

from __future__ import print_function
import apiclient
from httplib2 import Http
from oauth2client import file, client, tools
import pandas as pd

import json
import urllib.request
import pandas_bokeh

In [3]:
SPREADSHEET_ID = '1cgylzaHlX-v1KTTA7gxPO0-CbUT6FPqyDH93rqOgP9A'
RANGE_NAME = 'Sheet1'

In [4]:
# FUNCTIONS TO GET DATA FROM GOOGLE SPREADSHEET

def get_google_sheet(spreadsheet_id, range_name):
    """ Retrieve sheet data using OAuth credentials and Google Python API. """
    scopes = 'https://www.googleapis.com/auth/spreadsheets.readonly'
    # Setup the Sheets API
    store = file.Storage('credentials.json')
    creds = store.get()
    if not creds or creds.invalid:
        flow = client.flow_from_clientsecrets('client_secret.json', scopes)
        creds = tools.run_flow(flow, store)
    service = apiclient.discovery.build('sheets', 'v4', http=creds.authorize(Http()))

    # Call the Sheets API
    gsheet = service.spreadsheets().values().get(spreadsheetId=spreadsheet_id, range=range_name).execute()
    return gsheet


def gsheet2df(gsheet):
    """ Converts Google sheet data to a Pandas DataFrame.
    Note: This script assumes that your data contains a header file on the first row!
    Also note that the Google API returns 'none' from empty cells - in order for the code
    below to work, you'll need to make sure your sheet doesn't contain empty cells,
    or update the code to account for such instances.
    """
    header = gsheet.get('values', [])[0]   # Assumes first line is header!
    values = gsheet.get('values', [])[1:]  # Everything else is data.
    if not values:
        print('No data found.')
    else:
        all_data = []
        for col_id, col_name in enumerate(header):
            column_data = []
            for row in values:
                column_data.append(row[col_id])
            ds = pd.Series(data=column_data, name=col_name)
            all_data.append(ds)
        df = pd.concat(all_data, axis=1)
        return df

In [5]:
# GETTING DATA AND MAKING DATAFRAME

gsheet = get_google_sheet(SPREADSHEET_ID, RANGE_NAME)
df = gsheet2df(gsheet)

header_list = ['Date', 'Alderman', 'Bill_Number', 'Recusal_Reason', 'Subject', 'Title', 'Type_Of_Legislation']
df = df.reindex(columns = header_list)

In [7]:
# USING API AND APPENDING DATA

for index, row in df.iterrows():
    identifier = row['Bill_Number']
    api = 'https://ocd.datamade.us/bills/?identifier=' + identifier
    with urllib.request.urlopen(api) as url:
        data = json.loads(url.read().decode())
        
        subject = data['results'][0]['subject']
        title = data['results'][0]['title']
        classification = data['results'][0]['classification']

        df['Subject'][index] = subject
        df['Title'][index] = title
        df['Type_Of_Legislation'][index] = classification
        

        
df.to_csv('database.csv')

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/site-pac

_________

# CHART 1: RECUSALS BY DATE

In [124]:
# CHART 1: RECUSALS BY DATE; CLICK ON ALDERMAN

recusals_df = df.groupby(( "Date", "Alderman")).size().to_frame().reset_index()
recusals_df.rename(columns={0:"num_recusals"}, inplace=True)
recusals_df = recusals_df.pivot("Date", "Alderman", "num_recusals")
recusals_df = recusals_df.fillna(0)


recusals_df.plot_bokeh.bar(
    figsize =  (1300, 750),
    ylabel = "Number of Recusals",
    title = "Recusals by Date",
    alpha = 1)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Interpreting tuple 'by' as a list of keys, rather than a single key. Use 'by=[...]' instead of 'by=(...)'. In the future, a tuple will always mean a single key.
  This is separate from the ipykernel package so we can avoid doing imports until


Figure(id='6437', ...)

_______

# CHART 2: ALDERMAN BY TYPE OF RECUSAL

In [133]:
recusals_df = df.groupby(( "Alderman", "Recusal_Reason")).size().to_frame().reset_index()
recusals_df.rename(columns={0:"num_recusals"}, inplace=True)
recusals_df = recusals_df.pivot( "Alderman", "Recusal_Reason", "num_recusals")
recusals_df = recusals_df.fillna(0)


recusals_df.plot_bokeh.bar(
    figsize =  (1300, 750),
    ylabel = "Number of Recusals",
    title = "Recusals by Recusal_Reason",
    alpha = 1)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Interpreting tuple 'by' as a list of keys, rather than a single key. Use 'by=[...]' instead of 'by=(...)'. In the future, a tuple will always mean a single key.
  """Entry point for launching an IPython kernel.


Figure(id='7557', ...)

_________

In [146]:
df['Subject'] =  df['Subject'].map(lambda x: str(x)[:-2])
df['Subject'] =  df['Subject'].map(lambda x: str(x)[2:])

# CHART 3: Alderman Recusal by Subject

In [155]:
count = df.Subject.value_counts()
m = df.Subject.isin(count.index[count<3])
df.loc[m, 'Subject'] = 'Other'

In [165]:
# CHART 3: RECUSALS BY SUBJECT; CLICK ON Subject

recusals_df = df.groupby(("Alderman", "Subject")).size().to_frame().reset_index()
recusals_df.rename(columns={0:"num_recusals"}, inplace=True)
recusals_df = recusals_df.pivot("Alderman","Subject", "num_recusals")
recusals_df = recusals_df.fillna(0)


recusals_df.plot_bokeh.bar(
    figsize =  (1300, 750),
    ylabel = "Number of Recusals",
    title = "Recusals by Subject",
    alpha = 1)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Interpreting tuple 'by' as a list of keys, rather than a single key. Use 'by=[...]' instead of 'by=(...)'. In the future, a tuple will always mean a single key.
  This is separate from the ipykernel package so we can avoid doing imports until


Figure(id='10557', ...)

__________